<a href="https://colab.research.google.com/github/hamednasr/tensorflow/blob/main/04_transfer_learning_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# transfer learning in tensorflow: feature extraction

## load data

In [1]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-02-27 07:57:14--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   206MB/s    in 0.8s    

2022-02-27 07:57:15 (206 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
import zipfile
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

In [3]:
import os
for dirpath, dirnames,filenames in os.walk('10_food_classes_10_percent'):
  print(f'there are {len(dirnames)} folders and {len(filenames)} images in {dirpath}')

there are 2 folders and 0 images in 10_food_classes_10_percent
there are 10 folders and 0 images in 10_food_classes_10_percent/train
there are 0 folders and 75 images in 10_food_classes_10_percent/train/ice_cream
there are 0 folders and 75 images in 10_food_classes_10_percent/train/fried_rice
there are 0 folders and 75 images in 10_food_classes_10_percent/train/grilled_salmon
there are 0 folders and 75 images in 10_food_classes_10_percent/train/hamburger
there are 0 folders and 75 images in 10_food_classes_10_percent/train/chicken_curry
there are 0 folders and 75 images in 10_food_classes_10_percent/train/pizza
there are 0 folders and 75 images in 10_food_classes_10_percent/train/sushi
there are 0 folders and 75 images in 10_food_classes_10_percent/train/ramen
there are 0 folders and 75 images in 10_food_classes_10_percent/train/steak
there are 0 folders and 75 images in 10_food_classes_10_percent/train/chicken_wings
there are 10 folders and 0 images in 10_food_classes_10_percent/test


## preprocess the data

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_shape = (224, 224)
batch_size = 32

train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

train_datagen = ImageDataGenerator(rescale=1/255) 
test_datagen = ImageDataGenerator(rescale=1/255) 

print('Training images:')
train_data_10_percent = train_datagen.flow_from_directory(directory=train_dir,
                                                          target_size=image_shape,
                                                          batch_size=batch_size,
                                                          class_mode='categorical')
print('\nTest images:')
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                              target_size=image_shape,
                                              batch_size=batch_size,
                                              class_mode='categorical')

Training images:
Found 750 images belonging to 10 classes.

Test images:
Found 2500 images belonging to 10 classes.


## create a tensorboard callback

In [6]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  tensorboard_callback = tf.keras.callbakcs.tensorboard_callback(log_dir=log_dir)
  print(f'saving tensorboard log files to: {log_dir}')
  return tensorboard_callback

## create models using tensorflow hub

In [34]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, optimizers

In [7]:
# two models from tensorflow hub

resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5'
efficiennet_url='https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2'

In [30]:
def create_model(model_url, num_classes = 10):
  '''
  takes a tensorflow hub URL and creates a keras sequential model withit
  
  Args:
  model_url: the URL of model from tensorflow hub
  num_classes
  '''

  feature_extractor_layer = hub.KerasLayer(model_url,trainable=False,
                                           name='feature_extractor_layer',
                                           input_shape=image_shape+(3,))   
  
  model = tf.keras.Sequential([
                feature_extractor_layer,

                layers.Dense(num_classes, activation= 'softmax', name='output_layer')
      ])
  
  return model

### resnet model

In [31]:
resnet_model=create_model(resnet_url,train_data_10_percent.num_classes)

In [33]:
resnet_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_layer (Ke  (None, 2048)             42626560  
 rasLayer)                                                       
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 42,647,050
Trainable params: 20,490
Non-trainable params: 42,626,560
_________________________________________________________________


In [35]:
resnet_model.compile(loss = 'categorical_crossentropy',
                     optimizer = optimizers.Adam(),
                     metrics = ['accuracy'])